# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")
# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../World_Weather_Analysis/output_data")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,281.02,93,100,16.53,TF,1706380401
1,1,pisco,-13.7000,-76.2167,299.18,78,0,5.14,PE,1706380401
2,2,novodvinsk,64.4165,40.8122,269.01,91,100,3.95,RU,1706380402
3,3,waitangi,-43.9535,-176.5597,291.94,93,23,1.79,NZ,1706380402
4,4,college,64.8569,-147.8028,234.12,59,75,2.06,US,1706380306


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:


# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    scale = 2
)
# Display the map

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values

# Set parameters for Temp
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <300) & (city_data_df["Max Temp"] >250)]

# Set parameters for cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Set parameters for Humidity
city_data_df = city_data_df.loc[(city_data_df["Humidity"] < 80)]

# Drop any rows with null values

city_data_df = city_data_df.dropna()

# Display sample data

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,pisco,-13.7000,-76.2167,299.18,78,0,5.14,PE,1706380401
14,14,la rinconada,37.4873,-5.9826,292.83,64,0,1.03,ES,1706380404
36,36,murshidabad,24.1833,88.2667,286.63,51,0,2.16,IN,1706380408
41,41,kaoma,-14.7833,24.8000,292.38,65,0,2.18,ZM,1706380409
56,56,taoudenni,22.6783,-3.9836,295.29,12,0,4.35,ML,1706380412


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity


hotel_df = city_data_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
1,pisco,PE,-13.7000,-76.2167,
14,la rinconada,ES,37.4873,-5.9826,
36,murshidabad,IN,24.1833,88.2667,
41,kaoma,ZM,-14.7833,24.8000,
56,taoudenni,ML,22.6783,-3.9836,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel

radius = 10000
params = {"type" : "hotel",
          "keyword" : "hotel",
          "key" : "geoapify_key"
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
pisco - nearest hotel: No hotel found
la rinconada - nearest hotel: No hotel found
murshidabad - nearest hotel: No hotel found
kaoma - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel found
great falls - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
smara - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
illizi - nearest hotel: No hotel found
kaliganj - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
mhamid - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
dubbo - nearest hotel: No hotel found
truth or consequences - nearest hotel: No hotel found
darnah - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
stara pazova - nearest hotel: No hotel found
bismarck - nearest hotel: No hotel found
anchorage - nearest hotel: No hote

,City,Country,Lat,Lng,Hotel Name
1,pisco,PE,-13.7000,-76.2167,No hotel found
14,la rinconada,ES,37.4873,-5.9826,No hotel found
36,murshidabad,IN,24.1833,88.2667,No hotel found
41,kaoma,ZM,-14.7833,24.8000,No hotel found
56,taoudenni,ML,22.6783,-3.9836,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = .75,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)